# Test custom endpoints


In [1]:
from pprint import pprint

from osmocom.utils import b2h, i2h
from pySim.apdu import Apdu
from pySim.transport.pcsc import PcscSimLink
from smartcard.Exceptions import CardConnectionException

from resimulate.euicc.transport.pcsc_link import PcscLink

setup_apdus = ["01a4040010a06573746b6d65ffffffff667775706400", "0155555500"]
setup_adpus_v2 = ["01a4040010a06573746b6d65ffffffffffff6d677400"]
get_version_apdus = setup_apdus + ["aaff000008"]
test_apdus = setup_adpus_v2
v2_unlock_apdus = setup_apdus + [
    "aa21000000",
    "aa11000000",
    "aa12000000",
    "aa13000000",
    "aa00000000",
]
v1_unlock_apdus = setup_apdus + [
    "0021000000",
    "0011000000",
    "0012000000",
    "0013000000",
    "0000000000",
]


def str2ascii(data):
    return "".join([chr(int(data[i : i + 2], 16)) for i in range(0, len(data), 2)])


with PcscLink() as link:
    for apdu_str in get_version_apdus:
        apdu = Apdu(cmd=apdu_str)
        # pprint(
        #     f"APDU cla={b2h([apdu.cla])} ins={i2h([apdu.ins])} p1={i2h([apdu.p1])} p2={i2h([apdu.p2])} lc={i2h([apdu.lc])} data={b2h(apdu.cmd_data)} p3/le={i2h([apdu.p3])}"
        # )
        print(f"ASCII data: {str2ascii(apdu_str)}")
        print(f"APDU: {apdu_str}")
        try:
            resp, sw = link.send_apdu(apdu_str)
        except CardConnectionException as e:
            print(f"Card connection error: {e}")
            continue

        pprint(f"resp={resp or 'None'} sw={sw}")

    print(f"Final response: {str2ascii(resp)}")

ASCII data: ¤  estkmeÿÿÿÿfwupd 
APDU: 01a4040010a06573746b6d65ffffffff667775706400
'resp=None sw=9000'
ASCII data: UUU 
APDU: 0155555500
'resp=None sw=9000'
ASCII data: ªÿ 
APDU: aaff000008
'resp=5430303156303600 sw=9000'
Final response: T001V06 


In [1]:
import itertools

from tqdm.notebook import tqdm

values = [i for i in "0123456789abcdef"]

success_response = []

for i, j, k, l in tqdm(list(itertools.product(values, values, values, values))):
    with PcscSimLink(apdu_tracer=None) as link:
        for apdu_str in [f"{i}{j}{k}{l}0008"]:
            apdu = Apdu(cmd=apdu_str)
            try:
                resp, sw = link.send_apdu(apdu_str)
            except CardConnectionException as e:
                # print(f"Card connection error: {e}")
                continue

        if sw == "9000":
            success_response.append(apdu_str)
            # print(f"ASCII data: {str2ascii(apdu_str)}")
            pprint(f"apdu={apdu_str} resp={resp or 'None'} sw={sw}")

print(f"Final response: {str2ascii(resp)}")

  0%|          | 0/65536 [00:00<?, ?it/s]

NameError: name 'PcscSimLink' is not defined

# Mutate firmware blocks


In [ ]:
import os
import random

from pySim.transport.pcsc import PcscSimLink
from tqdm.auto import tqdm

BIN_PATH = "/home/niklas/Downloads/ESTKme-2024-3.4.3/files/T001V06-3.4.3"

SETUP_APDUS = ["01a4040010a06573746b6d65ffffffff667775706400", "0155555500"]
UNLOCK_APDU = "aa21000000"
CHECK_FLASH_STATUS_APDU = "aa13000000"
FINISH_FLASH_APDU = "aa00000000"

BLOCK_SIZE = 0x80D
MSS = 255


class MUTATION:
    BITFLIP = "bitflip"
    RANDOM_BYTE = "random_byte"
    ZERO_BLOCK = "zero_block"
    SHUFFLE_BLOCKS = "shuffle_blocks"
    TRUNCATE = "truncate"


class APDUPacket:
    def __init__(
        self,
        cla: int,
        ins: int,
        p1: int,
        p2: int,
        data: bytes = b"",
        le: int = 0,
    ):
        """Initializes an APDU packet."""
        self.cla = cla
        self.ins = ins
        self.p1 = p1
        self.p2 = p2
        self.data = data
        self.le = le

    @property
    def lc(self):
        return len(self.data) if self.data else 0

    def to_hex(self) -> str:
        """Returns the APDU packet as a hex string."""
        apdu = bytearray([self.cla, self.ins, self.p1, self.p2])
        if self.lc > 0:
            apdu.append(self.lc)
            apdu.extend(self.data)
        if self.le > 0:
            apdu.append(self.le)

        return apdu.hex()


def init_fwupd(link: PcscSimLink):
    for apdu_str in SETUP_APDUS:
        if not send(link, apdu_str):
            print("Setup failed!")


def send(link: PcscSimLink, apdu_str: str) -> bool:
    print(f"Sending {apdu_str}")
    resp, sw = link.send_apdu(apdu_str)
    print(f"Response: {sw} {resp}")
    return sw == "9000"


def send_program_block(
    link: PcscSimLink, data: bytes, mss: int, validate: bool = False
) -> int:
    remaining_bytes = len(data)
    ins = 0x12 if validate else 0x11

    while remaining_bytes > 0:
        chunk_size = min(mss, remaining_bytes)
        current_chunk = len(data) - remaining_bytes
        p1 = current_chunk >> 8
        p2 = current_chunk & 0xFF

        apdu = APDUPacket(cla=0xAA, ins=ins, p1=p1, p2=p2)

        if not validate:
            apdu.data = bytearray(data[current_chunk : current_chunk + chunk_size])
        else:
            apdu.le = chunk_size

        if not send(link, apdu.to_hex()):
            return False

        remaining_bytes -= chunk_size

    return True


def mutate_firmware(
    data: bytes,
    mutation_type: MUTATION = MUTATION.BITFLIP,
    mutation_rate: float = 0.01,
) -> bytes:
    print(f"Mutating firmware with {mutation_type} mutation")
    data = bytearray(data)
    length = len(data)

    match mutation_type:
        case MUTATION.BITFLIP:
            num_flips = max(1, int(length * mutation_rate))
            for _ in range(num_flips):
                index = random.randint(0, length - 1)
                bit = 1 << random.randint(0, 7)
                data[index] ^= bit

        case MUTATION.RANDOM_BYTE:
            num_mutations = max(1, int(length * mutation_rate))
            for _ in range(num_mutations):
                index = random.randint(0, length - 1)
                data[index] = random.randint(0, 255)

        case MUTATION.ZERO_BLOCK:
            start = random.randint(0, length - 20)
            end = min(length, start + random.randint(5, 20))
            for i in range(start, end):
                data[i] = 0x00

        case MUTATION.SHUFFLE_BLOCKS:
            block_size = 16
            num_blocks = length // block_size
            blocks = [
                data[i * block_size : (i + 1) * block_size] for i in range(num_blocks)
            ]
            random.shuffle(blocks)
            data = b"".join(blocks)

        case MUTATION.TRUNCATE:
            truncate_point = random.randint(int(length * 0.5), length)
            data = data[:truncate_point]

    return bytes(data)


def send_firmware(link: PcscSimLink, mss: int, mutation_type: MUTATION | None = None):
    program_block = 0

    total_size = os.path.getsize(BIN_PATH)
    num_blocks = (total_size // BLOCK_SIZE) + (1 if total_size % BLOCK_SIZE else 0)

    with (
        open(BIN_PATH, "rb") as file_stream,
        tqdm(total=num_blocks, desc="Flashing Firmware", unit="block") as pbar,
    ):
        while data := file_stream.read(BLOCK_SIZE):
            if mutation_type is not None and program_block % 3 == 0:
                data = mutate_firmware(data, mutation_type)

            transmit_success = send_program_block(link, data, mss)
            valid = send_program_block(link, data, mss, validate=True)

            if not transmit_success or not valid:
                print("⚠️ Flash failed!")
                return

            if not send(link, CHECK_FLASH_STATUS_APDU):
                print("⚠️ Flash status check failed!")
                return

            program_block += 1
            pbar.update(1)

    send(link, FINISH_FLASH_APDU)


with PcscSimLink(apdu_tracer=None) as link:
    print("init fwupd")
    init_fwupd(link)
    print("unlocking")
    send(link, UNLOCK_APDU)
    print(f"mss {MSS}")
    send_firmware(link, 128, mutation_type=None)
    print("done")

init fwupd
Sending 01a4040010a06573746b6d65ffffffff667775706400
Response: 9000 
Sending 0155555500
Response: 9000 
unlocking
Sending aa21000000


CardConnectionException: Failed to transmit with protocol T0. Transaction failed.: Transaction failed. (0x80100016)